In [1]:
import os
import sys
import re
import json                                       
import gnssrefl.gps as g
import gnssrefl.rinex2snr as rnx
import gnssrefl.quickLook_function as quick
import gnssrefl.gnssir as guts
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");

from datetime import datetime

bin_path = os.path.abspath(os.path.join('../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import check_parameters 
import environment

%matplotlib inline

In [ ]:
#Making sure environment variables are set
exists = environment.check_environment()
if exists == False:
    environment.set_environment()
else:
     print('environment variable ORBITS path is', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is', os.environ['EXE'])
        
# import the crx2rnx file
environment.download_crx2rnx()

In [ ]:
### Niwot Ridge, Colorado, USA

**Station name:** nwot

**Location:** [Niwot Ridge LTER](https://nwt.lternet.edu)

**Archive:** [UNAVCO](https://www.unavco.org)

**Ellipsoidal Coordinates:**

- Latitude: 40.05539 

- Longitude: -105.59053

- Height(m): 3522.729 

[UNAVCO station page](https://www.unavco.org/instrumentation/networks/status/nota/overview/NWOT)


### Niwot Ridge, Colorado, USA

**Station name:** nwot

**Location:** [Niwot Ridge LTER](https://nwt.lternet.edu)

**Archive:** [UNAVCO](https://www.unavco.org)

**Ellipsoidal Coordinates:**

- Latitude: 40.05539 

- Longitude: -105.59053

- Height(m): 3522.729 

[UNAVCO station page](https://www.unavco.org/instrumentation/networks/status/nota/overview/NWOT)


### Data Summary

Station nwot was originally installed/designed by Jim Normandeau (UNAVCO) to support GPS reflections research 
by Kristine Larson, Eric Small, Ethan Gutmann, Felipe Nievinski, and Mark Williams at the University of Colorado. 
The site was hosted by the Niwot Ridge LTER. 

NWOT was deliberately made to be taller than the typical geodetic antenna so that it would never be 
buried by snow. It is approximately 3 meters above 
the bare soil surface.  Because it was installed to support testing GPS reflections, NWOT has always tracked L2C.
nwot was also part of [PBO H2O](http://cires1.colorado.edu/portal/?station=nwot).

<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/NWOT.jpg" width=500/>

The site has generally not been used by geodesists and there is very little useful information 
about when data are available at either UNAVCO or Nevada Reno (i.e. no time series).
After the original receiver failed in spring 2015, a new receiver was installed in late 2016 by 
Mark Raleigh (then at CIRES, now at the University Oregon). Though the nwot receiver 
may be tracking now, it has not been downloaded in some time and there is no working telemetry.
We will focus on the data between 2009-2015.

### Make a SNR File and run quickLook

We will start by making a single SNR file. 
Here there are two options. The main archive for this dataset only provides the high-quality
L2C data in the highrate (1-sec) area. We do not need this sample rate for GPS reflectometry,
so to speed things up, we strongly encourage you to use the "special" archive option.  Here
the 1-sec data have been decimated to 15 seconds:

In [ ]:
station = 'nwot'
year = 2014 
doy = 270

lat = 40.055
long = -105.591
height = 3522.449

In [ ]:
# To understand what rinex2snr returns, you can uncomment the next line of code to learn more about this function 
# and it's default parameters
# check_parameters.rinex2snr?
args = check_parameters.rinex2snr(station, year, doy, archive='special', translator='hybrid')
rnx.run_rinex2snr(**args)

Both L1 and L2C signals can be used at this site, though the L2C data are far superior in quality
to the L1 data. Use this **quickLook** command to get a sense of the quality of the 
reflector height (RH) retrievals. First L1:

In [ ]:
# making a plotting function for the quicklook function
def quicklook_results(args, values):
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
    quadrants = ['NW', 'NE', 'SW', 'SE']
    axes = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

    for i, quadrant in enumerate(quadrants):
        satellites = values[quadrant].keys()
        fail_satellites = values[f'f{quadrant}'].keys()

        for failsat in fail_satellites:
            axes[i].plot(values[f'f{quadrant}'][failsat][0], values[f'f{quadrant}'][failsat][1], color='lightgrey') 
        for sat in satellites:
            axes[i].plot(values[quadrant][sat][0], values[quadrant][sat][1])

    ax[0,0].set_title('Northwest', size=14)
    ax[0,1].set_title('Northeast',size=14)
    ax[1,0].set_title('Southwest', size=14)
    ax[1,1].set_title('Southeast', size=14)

    for ax in axes:
        ax.set_xlabel('reflector height (m)', size=14)
        ax.set_ylabel('volts/volts', size=14)
        ax.grid()
    
    fig.suptitle(f'GNSS Station {args["station"].upper()}, {args["year"]} doy {args["doy"]}, freq L1, elevation angles {args["e1"]}-{args["e2"]} \n', size=16)
    fig.tight_layout()
    plt.show()
    
    
def quicklook_metrics(args, values):
#     fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
    quadrants = ['NW', 'NE', 'SW', 'SE']
    
    # re-organizing the data in a plotting friendly format
    success_data = {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise':[], 'Amplitude': []}
    fail_data =  {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise': [], 'Amplitude': []}
    
    for i, quadrant in enumerate(quadrants):
        for j in values[quadrant].keys():
            success_data['Azimuth'].append(datakeys[quadrant][j][0])
            success_data['Reflector Height'].append(datakeys[quadrant][j][1])
            success_data['Peak to Noise'].append(datakeys[quadrant][j][5])
            success_data['Amplitude'].append(datakeys[quadrant][j][4])
        for k in values[f'f{quadrant}'].keys():
            fail_data['Azimuth'].append(datakeys[f'f{quadrant}'][k][0])
            fail_data['Reflector Height'].append(datakeys[f'f{quadrant}'][k][1])
            fail_data['Peak to Noise'].append(datakeys[f'f{quadrant}'][k][5])
            fail_data['Amplitude'].append(datakeys[f'f{quadrant}'][k][4])

    return pd.DataFrame(success_data), pd.DataFrame(fail_data)   

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

In [ ]:
success, fail = quicklook_metrics(args, datakeys)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook Retrieval Metrics: {args["station"]} GPS L1', size=16)

for i, ax in enumerate(axes):
    g = sns.scatterplot(x='Azimuth',y=success.columns[i+1], data=success, ax=ax, label='good')
    g = sns.scatterplot(x='Azimuth',y=fail.columns[i+1], data=fail, ax=ax, color='lightgrey', label='bad')
    
# axes[0].legend(loc='upper right')
# avg_rh = np.mean(success['Reflector Height'])
# qc_val_peak2noise = round(min(success['Peak to Noise']))
# axes[1].axhline(qc_val_peak2noise, linestyle='--', color='black', label='QC value used')
# qc_val_amp = round(min(success['Amplitude']))
# axes[2].axhline(qc_val_amp, linestyle='--', color='black', label='QC value used')
print(f'Average reflector height value: {avg_rh:.1f}')
# print('QC value for peak to noise:', qc_val_peak2noise)
# print('QC value for amplitude:', qc_val_amp)

plt.tight_layout()
plt.show()

These periodograms are a bit ratty in the low RH area. There are 
nice strong peaks in the southern quadrants. Now try L2C:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=20)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

In [ ]:
success, fail = quicklook_metrics(args, datakeys)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook Retrieval Metrics: {args["station"]} GPS L1', size=16)

for i, ax in enumerate(axes):
    g = sns.scatterplot(x='Azimuth',y=success.columns[i+1], data=success, ax=ax, label='good')
    g = sns.scatterplot(x='Azimuth',y=fail.columns[i+1], data=fail, ax=ax, color='lightgrey', label='bad')
    
# axes[0].legend(loc='upper right')
# avg_rh = np.mean(success['Reflector Height'])
# qc_val_peak2noise = round(min(success['Peak to Noise']))
# axes[1].axhline(qc_val_peak2noise, linestyle='--', color='black', label='QC value used')
# qc_val_amp = round(min(success['Amplitude']))
# axes[2].axhline(qc_val_amp, linestyle='--', color='black', label='QC value used')
print(f'Average reflector height value: {avg_rh:.1f}')
# print('QC value for peak to noise:', qc_val_peak2noise)
# print('QC value for amplitude:', qc_val_amp)

plt.tight_layout()
plt.show()

### Make multiple years of SNR files 

We are going to look at the data from installation (Fall 2009) through Spring 2015. To speed things
up I will run 2009 and 2015 separately, while the year 2010 through 2014 can be analyzed in 
one line:

In [ ]:
args = check_parameters.rinex2snr(station, year=2009, doy=240, doy_end=365, archive='special', translator='fortran')
rnx.run_rinex2snr(**args)

args = check_parameters.rinex2snr(station, year=2010, doy=1, doy_end=366, archive='special', year_end=2014, translator='fortran')
rnx.run_rinex2snr(**args)

args = check_parameters.rinex2snr(station, year=2015, doy=1, doy_end=120, archive='special', translator='fortran')
rnx.run_rinex2snr(**args)

### Run gnssir for multiple years
Make a json file for your **gnssir** analysis:

In [ ]:
check_parameters.make_json(station, lat, long, height, e1=7, peak2noise=3.2, ampl=8)

I have opted to only use the southern quadrants (azimuths 90 through 270). Note that since
L5 was not tracked at this site, it is not listed in the json file. I am using a minimum elevation
angle of 7 degrees because this particular receiver had a limit on the number of satellites it
could track. In some cases this meant the low elevation data are not available and that triggers
QC restrictions.

In [ ]:
# This is the json file that was created
json_file = 'input/nwot.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    file['azval'] = [90, 180, 180, 270]
    file['freqs'] = [1,20]
    file['reqAmp'] = [8,8]
os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Once you have a json file set up, run gnssir for the years 2009-2015:

In [ ]:
check_parameters.gnssir?

In [ ]:
year = 2009
doy = 1
doy_end = 365
year_end = 2015 
plt=False
args = check_parameters.gnssir(station, year, doy, doy_end=doy_end, year_end=year_end, plt=plt, screenstats=False)
args
year_list = list(range(year, args['year_end'] + 1))
doy_list = list(range(doy, args['doy_end'] + 1))
for year in year_list:
    args['args']['year'] = year
    for doy in doy_list:
        args['args']['doy'] = doy
        guts.gnssir_guts(**args['args'])

## Compute daily average RH values
Use the daily_avg utility to compute RH for each day. A median filter of 0.25 meter is used
to eliminate large outliers and a minimum number of tracks is set to 10. This is relatively
low because of the small number of L2C transmitting satellites in the early years of
the dataset. The year inputs here are optional.

In [ ]:
check_parameters.daily_avg(station, medfilter=.25, ReqTracks=10, year1=2009, year2=2015, plt2screen=False, txtfile='nwot-dailyavg.txt')

In [ ]:
import matplotlib.pyplot as plt
def read_allRH_file(filepath, regex):
    data = {'dates': [], 'rh': []}
    #read daily average reflector heights
    with open(f'{refl_code_dir}{filepath}', 'r') as myfile:
        file = myfile.read()
        matches = re.finditer(regex, file, flags=re.MULTILINE)

        for match in matches:
            ydoy = f'{int(match.group("year"))}-{int(match.group("doy"))}'
            date = datetime.strptime(ydoy, '%Y-%j').date()
            data['dates'].append(date)
            data['rh'].append(float(match.group('rh')))
            
    return data

regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}_allRH.txt'
data = read_allRH_file(filepath, regex)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim()
g.set_ylabel('Reflector Height (m)');

In [ ]:
plt.figure(figsize=(8,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

In [ ]:
regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}-dailyavg.txt'
data = read_allRH_file(filepath, regex)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

In [ ]:
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(3.4,.5)
g.set_ylabel('Reflector Height (m)');

We installed the GPS site at Niwot Ridge because there was a long-standing experiment 
for measuring snow depth (and snow water equivalent). We therefore have a way to assess
accuracy. We download the *in situ* data from 
the [Niwot Ridge facility.](https://portal.edirepository.org/nis/mapbrowse?scope=knb-lter-nwt&identifier=34)
We will compare to pole 16, which is shown in the photograph above. 
The relevant Niwot Ridge csv file is provided here: 

[in situ data from the Niwot Ridge LTER](saddsnow.dw.data.csv)

If the daily average RH file created above is stored in the same directory 
as the Niwot Ridge *in situ* datafile, you can use 
[this python script](nwot_usecase.py) to visually compare them: